# Machine learning for the working (biomedical) researcher
## <center>ELIXIR-IIB Machine Learning for Biologists 2017</center>
### Marco Chierici
#### _Data scientist @ FBK/MPBA_


In this handout we will go through basic concepts of machine learning using Scikit-learn and the SEQC neuroblastoma data set [Zhang et al, _Genome Biology_, 2015].

<img src="img/zhang.png" width="65%" />

In particular, we will focus on **a subset of 272 samples (136 training, 136 test)**, aiming at predicting an **extreme disease outcome** (favorable vs unfavorable samples: see main paper).

<img src="img/zhang_tab2.png" width="95%" />

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Display plots within the notebook
%matplotlib inline

Load the training set data:

In [ ]:
data_tr = pd.read_csv("data/nb_train.txt.gz", dtype=str, sep='\t', index_col=0)

Now let's have a look at the data. First, the dimensions:

In [ ]:
data_tr.shape

What's inside?

A peek at the first rows reveals that the first column (the dataframe **index**) contains the sample IDs, the second column is the class (or target), and the remaining columns are genes:

In [ ]:
data_tr.head()

We can access the sample IDs...

In [ ]:
data_tr.index

... and the column names:

In [ ]:
data_tr.columns

For the remaining part of this hands-on, we need the data to be stored in a Numpy array:

In [ ]:
data_tr.values

This keeps the class column as well.

But...

We need to separate the class from the data, so let's recreate our array by dropping the class column from the original dataframe:

In [ ]:
data_tr.values[:, 1:]

Let's set the dtype to float and save the array to x_tr:

In [ ]:
x_tr = data_tr.values[:, 1:].astype(float)

When coding, it is a good practice to have a peek at the resulting variables, to be sure everything is OK: i.e., is that variable like it is supposed to be? Did I accidentally throw away a feature column?

This can avoid lots of problems later on!

In [ ]:
x_tr

Save the class column in another Numpy integer array:

In [ ]:
y_tr = data_tr["class"].values.astype(int)

In [ ]:
y_tr

---

### Quick recap

- y_tr = 1 indicates **unfavorable** neuroblastoma samples (**bad** outcome)
- y_tr = -1 indicates **favorable** neuroblastoma samples (**good** outcome)

---

Extract feature and sample names:

In [ ]:
feat_tr = data_tr.columns[1:].values.astype(str)
samp_tr = data_tr.index.values.astype(str)

In [ ]:
samp_tr

Now load the test set data in a Pandas dataframe and create the Numpy arrays for data and labels.

In [ ]:
# Your code here...

In [ ]:
%load sol1.py

In [ ]:
x_ts.shape

# Principal Component Analysis

As a first step, let's perform an **unsupervised learning** task on the data set "as is" by decomposing it in its Principal Components.

Instantiate a scikit-learn PCA object:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

Now we'll _fit_ the PCA object on the training data, at the same time _transforming_ them in the Principal Component space:

In [ ]:
z_tr = pca.fit_transform(x_tr)

Let's have a look at the _variance ratio_, i.e. the percentage of the variance explained by each component:

In [ ]:
print(pca.explained_variance_ratio_)

_What can you understand from these variance percentages? Could this task be "predictable" by some sort of model?_

Is it always convenient to visualize the first two principal components in a scatterplot, in order to get a first assessment of the goodness of the decomposition.

We will color the points in the plot according to our sample labels.

In [ ]:
plt.figure()
plt.scatter(z_tr[y_tr == 1, 0], z_tr[y_tr == 1, 1], color="r")
plt.scatter(z_tr[y_tr == -1, 0], z_tr[y_tr == -1, 1], color="b")
plt.title("PCA of Train data")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In a PCA plot, it is usually more informative to print also the explained variances (e.g., in the axis labels):

In [ ]:
vars = pca.explained_variance_ratio_
plt.figure()
plt.scatter(z_tr[y_tr == 1, 0], z_tr[y_tr == 1, 1], color="r")
plt.scatter(z_tr[y_tr == -1, 0], z_tr[y_tr == -1, 1], color="b")
plt.title("PCA of Train data")
plt.xlabel("PC1 (%.2f%%)" % (100*vars[0]))
plt.ylabel("PC2 (%.2f%%)" % (100*vars[1]))
plt.show()

_Now apply the transformation to the test data and plot it._

In [ ]:
# Your code here...

In [ ]:
%load sol2.py

# Supervised Learning

Based on the PCA we built on our data, we decide to try some supervised learning on them.

Scikit-learn provides you access to several models via a very convenient _fit_ and _predict_ interface.

For example, let's fit a Random Forest classifier on the whole training data and then use this model to predict the labels of the test data:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500) # instantiate a RandomForestClassifier with a custom parameter
clf.fit(x_tr, y_tr) # fit the model on training data

In [ ]:
y_pred = clf.predict(x_ts) # predict labels on test data

What if we wanted to try a Support Vector Machine instead?

In scikit-learn it is as easy as choosing the appropriate class, i.e. **SVC** (Support Vector Classification):

In [ ]:
from sklearn.svm import SVC

Then we can fit and predict by using the same code as above:

In [ ]:
clf = SVC() # instantiate a SVC object with default parameters
clf.fit(x_tr, y_tr)

In [ ]:
y_pred = clf.predict(x_ts)

_OK, I (over)simplified a bit here. In general, a classifier has hyper-**parameters** that need to be tuned. Default choices are not good in all situations._

_For example, in SVC the two main parameters are the **regularization** parameter C and the **kernel** function (linear, gaussian, polynomial, etc.)_

_Usually we search the parameter space (**Grid Search**) for the combination that yields the best Cross-Validation score_.

## Feature ranking

Let's go back to our previously trained Random Forest classifier.

In [ ]:
np.random.seed(42)
clf = RandomForestClassifier(n_estimators=500)
clf.fit(x_tr, y_tr)

What are the **most important features**? i.e., those having a greater power in discriminating the two sample classes?

One option is to rank features according to some classifier properties, such as Random Forest importances or Linear SVM weights.

We can sort the features according to the Random Forest importances (Gini impurity index), and make a simple stem plot of the first 10.

In [ ]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature ranking:")
for f in range(10):
    print("%d. feature %s (%f)" % (f + 1, feat_tr[indices[f]], importances[indices[f]]))


In [ ]:
plt.figure()
plt.title("Feature importances")
plt.stem(range(10), importances[indices[:10]], align="center")
plt.xticks(range(10), feat_tr[indices[:10]], rotation="vertical")
plt.xlim([-1, 10])
plt.show()

## Evaluating performance

Compute and print the confusion matrix.

### Recap

In this example, the first row is class -1, so the confusion matrix will look like:

|      |  |  Predicted  |    |
|------|-----------|----|----|
|      |           | -1 | 1  |
| True | -1        | TN | FP |
|      | 1         | FN | TP |


In [ ]:
y_pred = clf.predict(x_ts) # use the trained RandomForestClassifier to predict labels on x_ts

In [ ]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_ts, y_pred)
conf

The total number of class -1 test samples (AN = All Negatives) should be equal to the sum of the first row of the confusion matrix, i.e., TN + FP:

In [ ]:
np.sum(y_ts==-1) # total number of "class -1" samples in the test set

Similarly for class 1, i.e., AP = All Positives = TP + FN:

In [ ]:
np.sum(y_ts==1) # total number of "class +1" samples in the test set

Compute the Accuracy, remembering/using the formula: 

ACC = (TN + TP) / (TN + TP + FN + FP)

TN and TP are on the main diagonal of our conf Numpy array. The denominator is equivalent to the overall sample size.

In [ ]:
(conf[0,0] + conf[1,1])/y_ts.shape[0] # y_ts.shape[0] is the sample size of the test set

Luckily Scikit-learn provides a quite handy alternative:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_ts, y_pred)

Now compute the MCC (again, Scikit-learn comes to our help):

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_ts, y_pred)

Looks good!

But how do I know if this model performs similarly well on a similar/unseen data set?

In other words, does this model generalize beyond its training set?

This is why data partitioning techniques are used.

# Data partitioning

Simple random split of a dataset in two groups (hold-out strategy), leaving 25% of the samples for model evaluation:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_tr, y_tr, test_size=0.25, random_state=0)

## Cross-validation

Split the training dataset, x_tr, with a 5-fold partitioning schema, keeping the class label proportions across folds:

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

N = skf.get_n_splits(x_tr, y_tr)

for i, (idx_tr, idx_ts) in enumerate(skf.split(x_tr, y_tr)):
    print("Fold %d / %d" % (i+1, N))
    X_train, Y_train = x_tr[idx_tr], y_tr[idx_tr]
    X_test, Y_test = x_tr[idx_ts], y_tr[idx_ts]
    print("TRAIN size:", X_train.shape[0])
    print("-- class 1:", np.sum(Y_train==1), "class -1:", np.sum(Y_train==-1))
    print("TEST size:", X_test.shape[0])
    print("-- class 1:", np.sum(Y_test==1), "class -1:", np.sum(Y_test==-1))
    print()


## Implementing a basic Data Analysis Protocol

As a final example, we implement a 10x 5-fold Cross-validation schema with a simple feature ranking. 

For each CV iteration, a Random Forest model is trained on the training portion of the data, then features are ranked according to the Random Forest importances; a series of Random Forest models are built upon an increasing number of the ranked features (i.e., 1, 5, 10, etc.) and evaluated on the test data in terms of MCC.

The average MCC over the 10x5 CV iterations is computed for the different feature set sizes. We choose the feature set size that maximizes the average MCC.

This basic example is meant as a starting point for building more complex pipelines, i.e., with more feature steps, (bootstrapped) confidence intervals for MCC, computation of a unified ranked feature list (as in Jurman et al., _Bioinformatics_, 2008).

<img src="img/dap.png" width="85%">

In [ ]:
CV_N = 10 # number of CV iterations
CV_K = 5 # number of CV folds
FEATURE_STEPS = [1, 5, 10, 25, 50, 100]
# prepare output MCC array
MCC = np.empty((CV_K*CV_N, len(FEATURE_STEPS)))
         

In [ ]:
for n in range(CV_N):
    print("~~~ Iteration %d ~~~" % (n+1))
    skf = StratifiedKFold(n_splits=CV_K, shuffle=True, random_state=n)
    for i, (idx_tr, idx_ts) in enumerate(skf.split(x_tr, y_tr)):
        print("Fold %d" % (i+1))
        X_train, Y_train = x_tr[idx_tr], y_tr[idx_tr]
        X_test, Y_test = x_tr[idx_ts], y_tr[idx_ts]
        
        clf = RandomForestClassifier(n_estimators=500, random_state=n)
        clf.fit(X_train, Y_train)
        ranking = np.argsort( clf.feature_importances_ )[::-1]
        
        for j, s in enumerate(FEATURE_STEPS):
            v = ranking[:s] # consider the top s ranked features
            X_tr_fs, X_ts_fs = X_train[:, v], X_test[:, v] # extract them from internal train and test data
            clf.fit(X_tr_fs, Y_train) # train a classifier on the reduced train dataset
            YP = clf.predict(X_ts_fs) # predict on the reduced test dataset
            MCC[(n*CV_K)+i, j] = matthews_corrcoef(Y_test, YP) # evaluate the model performance
        
    print()


In [ ]:
np.save("MCC_CV", MCC)
# MCC = np.load("MCC_CV.npy")

In [ ]:
MCC_avg = np.mean(MCC, axis=0)
MCC_max = np.max(MCC_avg)
n_feats = FEATURE_STEPS[np.argmax(MCC_avg)]

In [ ]:
# average MCC for each feature step
for nf, mcc in zip(FEATURE_STEPS, MCC_avg):
    print("nf = %d, MCC = %.2f" % (nf, mcc))

print()

print("Best MCC = %.2f with %d features" % (MCC_max, n_feats))


In [ ]:
plt.figure()
plt.title("Average MCC")
plt.plot(FEATURE_STEPS, MCC_avg, 'o-')
plt.xlabel("Number of features")
plt.ylabel("MCC")
plt.show()